In [6]:
import os
import json
from PIL import Image
from glob import glob



def yolo_to_vgg(yolo_folder, images_folder, names, output_json):
    vgg_json = {}
    image_id = 0

    for yolo_path in sorted(glob(os.path.join(yolo_folder, "*.txt"))):
        # Get image path
        base = os.path.splitext(os.path.basename(yolo_path))[0]
        img_path = None
        for ext in [".jpg", ".jpeg", ".png"]:
            possible_path = os.path.join(images_folder, base + ext)
            if os.path.exists(possible_path):
                img_path = possible_path
                break

        if img_path is None:
            print(f"[WARN] No matching image found for {yolo_path}, skipping...")
            continue

        # Load image to get dimensions
        with Image.open(img_path) as img:
            width, height = img.size

        filename = os.path.basename(img_path)
        file_key = filename  # Unique key required by VIA

        # Initialize VGG entry for this image
        vgg_json[file_key] = {
            "filename": filename,
            "size": os.path.getsize(img_path),
            "regions": {},
            "file_attributes": {}
        }

        # Read YOLO labels
        with open(yolo_path, "r") as f:
            lines = f.read().strip().splitlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 7:
                continue  # Skip invalid lines

            class_id = int(parts[0])
            coords = list(map(float, parts[1:]))

            # Convert normalized coordinates to absolute pixels
            xs = [round(coords[i] * width) for i in range(0, len(coords), 2)]
            ys = [round(coords[i + 1] * height) for i in range(0, len(coords), 2)]

            # Create VGG region
            region = {
                "shape_attributes": {
                    "name": "polygon",
                    "all_points_x": xs,
                    "all_points_y": ys
                },
                "region_attributes": {
                    "label": names[class_id]
                }
            }
            vgg_json[file_key]["regions"][f"{len(vgg_json[file_key]['regions'])}"] = region

    # Save to JSON
    with open(output_json, "w") as out:
        json.dump(vgg_json, out, indent=2)

    print(f"[INFO] Conversion complete → {output_json}")


# ------------------- USAGE -------------------

NAMES = {0: 'beige_boucharde_degrade',
 1: 'beige_boucharde_satisfaisant',
 2: 'beige_flamme_degrade',
 3: 'beige_flamme_satisfaisant',
 4: 'beige_spuntato_degrade',
 5: 'beige_spuntato_satisfaisant',
 6: 'bleu_boucharde_degrade',
 7: 'bleu_boucharde_satisfaisant',
 8: 'bleu_flamme_degrade',
 9: 'bleu_flamme_satisfaisant',
 10: 'bleu_spuntato_degrade',
 11: 'bleu_spuntato_satisfaisant'}

# Path to YOLO segmentation labels (.txt files)
YOLO_LABELS_DIR = "Stanislas/datasetv1/val/labels"
# Path to images
IMAGES_DIR = "Stanislas/datasetv1/val/images"
# Output VGG JSON file
OUTPUT_JSON = "Stanislas/datasetv1/val/vgg_annotations.json"

yolo_to_vgg(YOLO_LABELS_DIR, IMAGES_DIR, NAMES, OUTPUT_JSON)


[INFO] Conversion complete → Stanislas/datasetv1/val/vgg_annotations.json
